##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/federated/tutorials/building_your_own_federated_learning_algorithm"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/federated/tutorials/building_your_own_federated_learning_algorithm.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/federated/tutorials/building_your_own_federated_learning_algorithm.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/federated/tutorials/building_your_own_federated_learning_algorithm.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar  bloc de notas</a>
</td>
</table>

## Antes de empezar

Antes de empezar, ejecute lo que se encuentra a continuación, para asegurarse de que el entorno esté preparado correctamente. Si no ve un mensaje de inicio, para más instrucciones, consulte la guía de [instalación](../install.md). 

In [ ]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated

In [ ]:
import tensorflow as tf
import tensorflow_federated as tff

**NOTA**: Esta colaboración ha sido verificada para trabajar con la [versión de lanzamiento más reciente](https://github.com/tensorflow/federated#compatibility) del paquete pip `tensorflow_federated`, pero el proyecto federado de TensorFlow aún se encuentra en una etapa de desarrollo previa al lanzamiento. Por lo tanto, es probable que no funcione en `main`.

# Creación de un algoritmo propio de aprendizaje federado

Con los tutoriales de [clasificación de imágenes](federated_learning_for_image_classification.ipynb) y de [generación de textos](federated_learning_for_text_generation.ipynb), se aprende a preparar el modelo y las canalizaciones de los datos para el aprendizaje federado (FL, por sus siglas en inglés) y se realizan entrenamientos federados mediante la capa de la API `tff.learning` de TFF.

Es la punta del iceberg en la investigación sobre el aprendizaje federado. En este tutorial se analiza cómo implementar los algoritmos de aprendizaje federado *sin* delegar a la API `tff.learning`. Con este tutorial, logrará lo siguiente:

**Objetivos:**

- Entender la estructura general de los algoritmos de aprendizaje federado.
- Explorar el *núcleo federado* de TFF.
- Usar el núcleo federado para implementar directamente el cálculo del promedio federado.

Si bien este tutorial contiene todo lo necesario para entenderlo sin lecturas extra, puede ser de utilidad consultar primero los tutoriales de [clasificación de imágenes](federated_learning_for_image_classification.ipynb) y de [generación de textos](federated_learning_for_text_generation.ipynb).


## Preparación de los datos de entrada

Primero, hay que cargar y procesar el conjunto de datos EMNIST incluido en TFF. Para más detalles, consulte el tutorial sobre [clasificación de imágenes](federated_learning_for_image_classification.ipynb).

In [ ]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

A fin de alimentar nuestro modelo con el conjunto de datos, estos datos se aplanan. Cada ejemplo se transforma en una tupla con la forma `(flattened_image_vector, label)`.

In [ ]:
NUM_CLIENTS = 10
BATCH_SIZE = 20

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch of EMNIST data and return a (features, label) tuple."""
    return (tf.reshape(element['pixels'], [-1, 784]), 
            tf.reshape(element['label'], [-1, 1]))

  return dataset.batch(BATCH_SIZE).map(batch_format_fn)

Ahora, seleccione una pequeña cantidad de clientes y aplique el preprocesamiento anterior a los conjuntos de datos.

In [ ]:
client_ids = sorted(emnist_train.client_ids)[:NUM_CLIENTS]
federated_train_data = [preprocess(emnist_train.create_tf_dataset_for_client(x))
  for x in client_ids
]

## Preparación del modelo

Se usa el mismo modelo que en el tutorial de [clasificación de imágenes](federated_learning_for_image_classification.ipynb). En este modelo (implementado mediante `tf.keras`) hay una sola capa oculta, seguida por una capa <em>softmax</em>.

In [ ]:
def create_keras_model():
  initializer = tf.keras.initializers.GlorotNormal(seed=0)
  return tf.keras.models.Sequential([
      tf.keras.layers.Input(shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer=initializer),
      tf.keras.layers.Softmax(),
  ])

A fin de usar este modelo en TFF, encapsule el modelo Keras como un [`tff.learning.models.VariableModel`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model). Esto permite hacer el [pase hacia adelante](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model#forward_pass) del modelo dentro de TFF y [extraer las salidas](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model#report_local_unfinalized_metrics) (del modelo). Para más información, también consulte el tutorial sobre [clasificación de imágenes](federated_learning_for_image_classification.ipynb).

In [ ]:
def model_fn():
  keras_model = create_keras_model()
  return tff.learning.models.from_keras_model(
      keras_model,
      input_spec=federated_train_data[0].element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

Mientras que en el caso de arriba se usó `tf.keras` para crear un `tff.learning.models.VariableModel`, TFF admite modelos mucho más generales. Estos modelos tienen los siguientes atributos relevantes que capturan los pesos:

- `trainable_variables`: un iterable de tensores correspondiente a las capas entrenables.
- `non_trainable_variables`: un iterable de tensores correspondiente a capas no entrenables.

Para este tutorial, solamente se usarán las `trainable_variables`. (ya que son las únicas que tiene el modelo).

# Creación de un algoritmo propio de aprendizaje federado

Si bien la API `tff.learning` permite que uno cree muchas variantes del cálculo del promedio federado, hay otros algoritmos federados que no se adaptan perfectamente a este marco de trabajo. Por ejemplo, tal vez le convenga agregar algoritmos de regularización, recorte (<em>clipping</em>) u otros más complicados como el [entrenamiento GAN federado](https://github.com/tensorflow/federated/tree/main/tensorflow_federated/python/research/gans). Probablemente, por otra parte, lo que le resulte interesante sea el [análisis federado](https://ai.googleblog.com/2020/05/federated-analytics-collaborative-data.html).

Si pretende trabajar con algoritmos más avanzados, deberá escribir su propio algoritmo con TFF. En muchos casos, los algoritmos federados tienen los siguientes 4 componentes:

1. Un paso para la emisión (<em>broadcast</em>) del servidor al cliente.
2. Un paso para la actualización del cliente local.
3. Un paso para la carga del cliente al servidor.
4. Un paso para la actualización del servidor.

En TFF, un algoritmo federado, normalmente, está representado por un [`tff.templates.IterativeProcess`](https://www.tensorflow.org/federated/api_docs/python/tff/templates/IterativeProcess) (que será referido simplemente como un `IterativeProcess`). Es una clase que contiene las funciones `initialize` y `next`. Aquí, `initialize` se usa para inicializar el servidor y `next` realizará una ronda de comunicación del algoritmo federado. Escribamos un esquema sobre cómo debería lucir de nuestro proceso iterativo para FedAvg.

Primero, hay una función para inicializar que simplemente crea `tff.learning.models.VariableModel` y devuelve sus pesos entrenables.

In [ ]:
def initialize_fn():
  model = model_fn()
  return model.trainable_variables

Esta función tiene buen aspecto, pero como verá más adelante, deberá hacerle una pequeña modificación para convertirla en un "cálculo TFF".

Luego, esbocemos un `next_fn`.

In [ ]:
def next_fn(server_weights, federated_dataset):
  # Broadcast the server weights to the clients.
  server_weights_at_client = broadcast(server_weights)

  # Each client computes their updated weights.
  client_weights = client_update(federated_dataset, server_weights_at_client)

  # The server averages these updates.
  mean_client_weights = mean(client_weights)

  # The server updates its model.
  server_weights = server_update(mean_client_weights)

  return server_weights

Centrémonos en implementar estos cuatro componentes por separado. Primero, enfoquémonos en las partes que se pueden implementar en TensorFlow puro, a saber, los pasos relacionados con el cliente y el servidor.


## Bloques de TensorFlow 

### Actualización del cliente

El `tff.learning.models.VariableModel` se puede usar para hacer el entrenamiento del cliente, esencialmente, del mismo modo en que se entrenaría un modelo de TensorFlow. En particular, uno puede usar `tf.GradientTape` para calcular el gradiente en lotes de datos y luego aplicarlo con un `client_optimizer`. Este procedimiento solamente incluirá los pesos entrenables.


In [ ]:
@tf.function
def client_update(model, dataset, server_weights, client_optimizer):
  """Performs training (using the server model weights) on the client's dataset."""
  # Initialize the client model with the current server weights.
  client_weights = model.trainable_variables
  # Assign the server weights to the client model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        client_weights, server_weights)

  # Use the client_optimizer to update the local model.
  for batch in dataset:
    with tf.GradientTape() as tape:
      # Compute a forward pass on the batch of data
      outputs = model.forward_pass(batch)

    # Compute the corresponding gradient
    grads = tape.gradient(outputs.loss, client_weights)
    grads_and_vars = zip(grads, client_weights)

    # Apply the gradient using a client optimizer.
    client_optimizer.apply_gradients(grads_and_vars)

  return client_weights

### Actualización del servidor

La actualización del servidor FedAvg es más simple que la del cliente. En este tutorial implementaremos el cálculo de promedios federados "vainilla", en el que los pesos del modelo del servidor se reemplazan con el promedio de los pesos del modelo del cliente. Una vez más, solamente se usan los pesos entrenables.

In [ ]:
@tf.function
def server_update(model, mean_client_weights):
  """Updates the server model weights as the average of the client model weights."""
  model_weights = model.trainable_variables
  # Assign the mean client weights to the server model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        model_weights, mean_client_weights)
  return model_weights

El fragmento se podría simplificar sencillamente con la devolución de `mean_client_weights`. Sin embargo, en las implementaciones más avanzadas del cálculo de promedio federado se usa `mean_client_weights` con técnicas más sofisticadas como <em>momentum</em> o adaptabilidad.

**Desafío**: implementar una versión `server_update` que actualice los pesos del servidor para ser el punto medio entre model_weights y mean_client_weights. (Nota: este tipo de método de "punto medio" es análogo a trabajos recientes sobre el [<em>optimizador Lookahead</em>](https://arxiv.org/abs/1907.08610)).

Hasta el momento, solamente se ha incluido código de TensorFlow. El motivo es el diseño, ya que el TFF permite usar gran parte del código de TensorFlow con el que ya está familiarizado. A continuación, deberá especificar la **lógica de orquestación**; es decir, la que dicta que el servidor emite (<em>broadcast</em>) al cliente y que el cliente carga en el servidor.

El *núcleo federado* de TFF será indispensable.

# Introducción al núcleo federado

El núcleo federado (FC, por sus siglas en inglés) es un conjunto de interfaces de bajo nivel que sirve como base para la API `tff.learning`. Sin embargo, estas interfaces no se limitan al aprendizaje. De hecho, se pueden usar para análisis y muchos otros cálculos de datos distribuidos.

A un alto nivel, el núcleo federado es un entorno de desarrollo que permite expresar de manera compacta la lógica de programación para combinar código de TensorFlow con los operadores de comunicación distribuidos (como las sumas y las emisiones distribuidas). El objetivo es brindarles a los investigadores y especialistas el control explícito de la comunicación distribuida en sus sistemas, sin requerir de otros detalles para la implementación (tales como la especificación de los intercambios de mensajes de red punto a punto).

Un punto clave es que TFF está diseñado para la preservación de la privacidad. Por lo tanto, permite el control explícito del sitio donde residen los datos, para prevenir la acumulación indeseada de datos en el lugar del servidor centralizado.

## Datos federados

El concepto de los "datos federados" es clave en TFF. Se refiere a una colección de elementos de datos alojados en un grupo de dispositivos en un sistema distribuido (p. ej., las bases de datos de clientes o los pesos del modelo del servidor). La colección entera de valores de todos los dispositivos se representa con un solo *valor federado*.

Por ejemplo, supongamos que hay dispositivos clientes y que cada uno tiene un flotante que representa la temperatura de un tensor. Esos flotantes se pueden representar como *flotante federado* de la siguiente manera:

In [ ]:
federated_float_on_clients = tff.FederatedType(tf.float32, tff.CLIENTS)

Los tipos federados son especificados por un tipo de `T` de los miembros que lo componen (p. ej., `tf.float32`) y un grupo de dispositivos `G`. Normalmente, `G` es `tff.CLIENTS` o `tff.SERVER`. Un tipo federado como tal se representa con `{T}@G`, tal como se muestra a continuación.

In [ ]:
str(federated_float_on_clients)

'{float32}@CLIENTS'

¿Por qué a TFF le interesan tanto las ubicaciones? El objetivo clave de TFF es el de facilitar la escritura de código que se podría implementar en un sistema distribuido real. Significa que es vital razonar con respecto a qué subconjuntos de dispositivos ejecutan qué códigos y dónde residen las diferentes porciones de datos.

TFF se centra en tres cosas: en los *datos*, en dónde se *ubican* los datos y en cómo se *transforman* esos datos. Las primeras dos se encuentran encapsuladas dentro de los tipos federados, mientras que la última, en *cálculos federados*.

## Cálculos federados

TFF es un entorno de programación funcional fuertemente tipado cuyas unidades básicas son *cálculos federados*. Son porciones de lógica que aceptan valores federados como entrada y devuelven valores federados como salida.

Por ejemplo, supongamos que quisiera calcular el promedio de temperaturas en los sensores de nuestro cliente. Podría definir lo siguiente (con nuestro flotante federado):

In [ ]:
@tff.federated_computation(tff.FederatedType(tf.float32, tff.CLIENTS))
def get_average_temperature(client_temperatures):
  return tff.federated_mean(client_temperatures)

Podría preguntarse, en qué difiere esto del decorador `tf.function` de TensorFlow. La respuesta determinante es que el código generado por `tff.federated_computation` no es un código de TensorFlow ni de Python. Es una especificación de un sistema distribuido en un *lenguaje pegamento* interno independiente de plataformas.

Si bien es cierto que puede sonar complicado, puede pensar en los cálculos TFF como funciones con firmas bien definidas. Este tipo de firmas se puede consultar directamente.

In [ ]:
str(get_average_temperature.type_signature)

'({float32}@CLIENTS -> float32@SERVER)'

Este `tff.federated_computation` acepta argumentos del tipo federado `<float>@CLIENTS` y devuelve valores del mismo tipo `<float>@SERVER`. Los cálculos federados también van de servidor a cliente, de cliente a cliente o de servidor a servidor. Los cálculos federados además se pueden componer como las funciones normales, siempre y cuando haya coincidencia entre las firmas de tipo.

Para facilitar el desarrollo, TFF permite invocar un `tff.federated_computation` como una función Python. Por ejemplo, se puede llamar lo siguiente:

In [ ]:
get_average_temperature([68.5, 70.3, 69.8])

69.53334

## Los cálculos sin ejecución <em>eager</em> y TensorFlow

Hay dos restricciones fundamentales para tener en cuenta. La primera, es que cuando un interpretador Python encuentra un decorador `tff.federated_computation`, la función se rastrea una vez y se serializa para futuros usos. Debido a la naturaleza descentralizada del aprendizaje federado, este uso futuro puede producirse en cualquier otro lugar, como en un entorno de ejecución remota. Por lo tanto, los cálculos TFF son fundamentalmente *non-eager* (no utilizan ejecución *eager*. Este comportamiento es, en cierto modo, análogo al del decorador [`tf.function`](https://www.tensorflow.org/api_docs/python/tf/function) en TensorFlow.

La segunda, es que un cálculo federado solamente puede estar compuesto por operadores federados ( como `tff.federated_mean`), no pueden contener operaciones de TensorFlow. Hay que confinar el código de TensorFlow a bloques decorados con `tff.tf_computation`. El código TensorFlow más común, directamente, se puede decorar, como la siguiente función que toma un número y le agrega `0.5`.

In [ ]:
@tff.tf_computation(tf.float32)
def add_half(x):
  return tf.add(x, 0.5)

Estas también son firmas de tipo, pero *sin ubicaciones*. Por ejemplo, se puede llamar lo siguiente:

In [ ]:
str(add_half.type_signature)

'(float32 -> float32)'

De este modo se pone de manifiesto la gran diferencia que hay entre `tff.federated_computation` y `tff.tf_computation`. El primero tiene ubicaciones explícitas, mientras que el segundo no.

Se pueden usar bloques `tff.tf_computation` en cálculos federados para ubicaciones específicas. Cree una función que agregue un medio (<em>add half</em>), pero solamente a flotantes federados de clientes. Se puede hacer con `tff.federated_map`, que aplica un `tff.tf_computation` dado y, a la vez, preserva la ubicación.

In [ ]:
@tff.federated_computation(tff.FederatedType(tf.float32, tff.CLIENTS))
def add_half_on_clients(x):
  return tff.federated_map(add_half, x)

Esta función es casi idéntica a `add_half`, excepto porque solamente acepta valores con ubicación en `tff.CLIENTS` y devuelve valores con la misma ubicación. Esto se puede observar en su firma de tipo:

In [ ]:
str(add_half_on_clients.type_signature)

'({float32}@CLIENTS -> {float32}@CLIENTS)'

En resumen:

- TFF opera sobre valores federados.
- Cada valor federado tiene un *tipo federado*, con un *tipo* (p. ej., `tf.float32`) y una *ubicación* (p. ej., `tff.CLIENTS`).
- Los valores federados se pueden transformar con *cálculos federados*, que se deben decorar con `tff.federated_computation` y una firma de tipo federado.
- El código TensorFlow debe estar contenido en bloques con decoradores `tff.tf_computation`.
- Estos bloques, después se pueden incorporar en cálculos federados.


# Creación de un algoritmo propio de aprendizaje federado (repaso)

Ahora que ya tiene una idea de lo que es el núcleo federado, puede crear su propio algoritmo de aprendizaje federado. Recuerde que antes (arriba) ya definió un `initialize_fn` y `next_fn` para su algoritmo. El `next_fn` usará `client_update` y `server_update` que ya ha definido<br>con código de TensorFlow puro.

Sin embargo, para hacer nuestro algoritmo con un cálculo federado, necesitará que tanto `next_fn` como `initialize_fn` sean cada uno un `tff.federated_computation`.

## Bloques federados de TensorFlow 

### Creación del cálculo de inicialización

La función de inicializar será bastante simple: deberá crear un modelo con `model_fn`. Sin embargo, recuerde que debe separar nuestro código de TensorFlow con `tff.tf_computation`.

In [ ]:
@tff.tf_computation
def server_init():
  model = model_fn()
  return model.trainable_variables

Entonces, ahora, puede pasarlo directamente a cálculo federado con `tff.federated_value`.

In [ ]:
@tff.federated_computation
def initialize_fn():
  return tff.federated_value(server_init(), tff.SERVER)

### Creación de `next_fn`

El código de actualización de cliente y servidor ahora se puede usar para escribir el algoritmo real. Primero, se transformará el `client_update` en un `tff.tf_computation` que acepta un conjunto de datos del cliente y los pesos del servidor, y sale un tensor de pesos del cliente actualizado.

Necesitará los tipos correspondientes que decoren adecuadamente nuestra función. Afortunadamente, el tipo de pesos del servidor se puede extraer directamente desde nuestro modelo.

In [ ]:
whimsy_model = model_fn()
tf_dataset_type = tff.SequenceType(whimsy_model.input_spec)

Observemos la firma de tipo del conjunto de datos. Recordemos que tomamos imágenes de 28 por 28 (con etiquetas de enteros) y las aplanamos.

In [ ]:
str(tf_dataset_type)

'<float32[?,784],int32[?,1]>*'

También se puede extraer el tipo de pesos del modelo con nuestra función `server_init`, que figura más arriba.

In [ ]:
model_weights_type = server_init.type_signature.result

Al examinar la firma de tipo, podrá ver la arquitectura del modelo.

In [ ]:
str(model_weights_type)

'<float32[784,10],float32[10]>'

Ahora podemos crear nuestro propio `tff.tf_computation` para la actualización del cliente.

In [ ]:
@tff.tf_computation(tf_dataset_type, model_weights_type)
def client_update_fn(tf_dataset, server_weights):
  model = model_fn()
  client_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
  return client_update(model, tf_dataset, server_weights, client_optimizer)

La versión `tff.tf_computation` de la actualización del servidor se puede definir de un modo similar, con los tipos que ya ha extraído.

In [ ]:
@tff.tf_computation(model_weights_type)
def server_update_fn(mean_client_weights):
  model = model_fn()
  return server_update(model, mean_client_weights)

Por último, pero no menos importante, deberá crear el `tff.federated_computation` que une todo. Esta función aceptará dos *valores federados*, uno correspondiente a los pesos del servidor (con la ubicación `tff.SERVER`) y otro correspondiente a los conjuntos de datos del cliente (con la ubicación `tff.CLIENTS`).

Tenga en cuenta que ambos tipos ya han sido definidos más arriba. Simplemente debe darles la ubicación adecuada con `tff.FederatedType`.

In [ ]:
federated_server_type = tff.FederatedType(model_weights_type, tff.SERVER)
federated_dataset_type = tff.FederatedType(tf_dataset_type, tff.CLIENTS)

¿Recuerda los 4 elementos de un algoritmo FL (de aprendizaje federado)?

1. Un paso para la emisión (<em>broadcast</em>) del servidor al cliente.
2. Un paso para la actualización del cliente local.
3. Un paso para la carga del cliente al servidor.
4. Un paso para la actualización del servidor.

Ahora que ha creado lo anterior, cada parte se puede representar de forma compacta como una sola línea de código TFF. Esta simplicidad es el motivo por el cual ha debido prestar suma atención a la especificación de cosas como los tipos federados.

In [ ]:
@tff.federated_computation(federated_server_type, federated_dataset_type)
def next_fn(server_weights, federated_dataset):
  # Broadcast the server weights to the clients.
  server_weights_at_client = tff.federated_broadcast(server_weights)

  # Each client computes their updated weights.
  client_weights = tff.federated_map(
      client_update_fn, (federated_dataset, server_weights_at_client))
  
  # The server averages these updates.
  mean_client_weights = tff.federated_mean(client_weights)

  # The server updates its model.
  server_weights = tff.federated_map(server_update_fn, mean_client_weights)

  return server_weights

Ahora tiene un `tff.federated_computation` tanto para la inicialización del algoritmo como para la ejecución de un paso del algoritmo. Para terminarlo, debe pasar estos elementos a `tff.templates.IterativeProcess`.

In [ ]:
federated_algorithm = tff.templates.IterativeProcess(
    initialize_fn=initialize_fn,
    next_fn=next_fn
)

Observemos la *firma de tipo * de las funciones `initialize` y `next` de nuestro proceso iterativo.

In [ ]:
str(federated_algorithm.initialize.type_signature)

'( -> <float32[784,10],float32[10]>@SERVER)'

Refleja el hecho de que `federated_algorithm.initialize` es una función no argumentativa que devuelve un modelo de una sola capa (con una matriz de peso de 784 por 10, y 10 unidades de sesgo).

In [ ]:
str(federated_algorithm.next.type_signature)

'(<server_weights=<float32[784,10],float32[10]>@SERVER,federated_dataset={<float32[?,784],int32[?,1]>*}@CLIENTS> -> <float32[784,10],float32[10]>@SERVER)'

Aquí, uno puede ver que `federated_algorithm.next` acepta un modelo de servidor y datos del cliente, y devuelve un modelo de servidor actualizado.

## Evaluación del algoritmo

Ejecutemos algunas rondas y veamos cómo cambia la pérdida. Primero, definirá una función de evaluación con el modo *centralizado* referido en el segundo tutorial.

En primer lugar, creará un conjunto de datos de evaluación centralizado y luego aplicará el mismo preprocesamiento que se usó para los datos entrenados.

In [ ]:
central_emnist_test = emnist_test.create_tf_dataset_from_all_clients()
central_emnist_test = preprocess(central_emnist_test)

A continuación, deberá escribir una función que acepte un estado del servidor y use Keras para evaluar en el conjunto de datos de prueba. Si está familiarizado con `tf.Keras`, todo esto también le resultará familiar; de todos modos, preste particular atención al uso de `set_weights`.

In [ ]:
def evaluate(server_state):
  keras_model = create_keras_model()
  keras_model.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]  
  )
  keras_model.set_weights(server_state)
  keras_model.evaluate(central_emnist_test)

Ahora, inicialicemos nuestro algoritmo y evaluemos el conjunto de prueba.

In [ ]:
server_state = federated_algorithm.initialize()
evaluate(server_state)

2042/2042 [==============================] - 2s 767us/step - loss: 2.8479 - sparse_categorical_accuracy: 0.1027


Entrenemos durante algunas rondas y veamos si cambia algo.

In [ ]:
for round in range(15):
  server_state = federated_algorithm.next(server_state, federated_train_data)

In [ ]:
evaluate(server_state)

2042/2042 [==============================] - 2s 738us/step - loss: 2.5867 - sparse_categorical_accuracy: 0.0980


Hay una disminución leve en la función de pérdida. Si bien el salto es pequeño, solamente ha realizado 15 rondas de entrenamiento y sobre un subconjunto reducido de clientes. Para ver mejores resultados, probablemente deba hacer cientos o miles de rondas.

## Modificación del algoritmo

En este punto, detengámonos a pensar sobre lo que hemos logrado. Ha implementado el cálculo promedio federado directamente mediante la combinación de código de TensorFlow puro (para las actualizaciones del cliente y del servidor) con cálculos federados del núcleo federado de TFF.

Para realizar un aprendizaje más sofisticado, simplemente puede alterar lo que hizo arriba. En particular, editando el código de TF puro mencionado puede cambiar la manera en que el cliente realiza el entrenamiento o cómo el servidor actualiza su modelo.

**Desafío:** agregar [recorte (<em>clipping</em>) de gradiente](https://towardsdatascience.com/what-is-gradient-clipping-b8e815cdfb48) a la función `client_update`.


Si lo que busca es hacer cambios más grandes, también podría hacer que el servidor almacene y emita más datos. Por ejemplo, el servidor también podría almacenar la velocidad de aprendizaje del cliente y hacerla decaer a lo largo del tiempo. Tenga en cuenta que, para que esto suceda, podría haber que hacer cambios en las firmas de tipo usadas arriba en las llamadas de `tff.tf_computation`.

**Desafío mayor:** implementar el cálculo de promedio federado con el decaimiento de la velocidad en los clientes.

A esta altura, podría empezar a darse cuenta de cuánta flexibilidad hay en lo que puede implementar en este marco de trabajo. Para más ideas (incluida la respuesta al desafío mayor anterior) puede ver el código fuente para [`tff.learning.algorithms.build_weighted_fed_avg`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_avg) o consultar varios [proyectos de investigación](https://github.com/google-research/federated) en los que se usa TFF.